In [72]:
%env CYPHER=/usr/bin/cypher-shell
%env HOST={uri}
%env USER=neo4j
%env PW={password}

env: CYPHER=/usr/bin/cypher-shell
env: HOST={uri}
env: USER=neo4j
env: PW={password}


In [4]:
CONNECT="$CYPHER -a $HOST -u $USER -p $PW"

In [7]:
from IPython.core.magic import register_cell_magic
import shlex

@register_cell_magic('neo')
def neo(line, cell):
    !{CONNECT} {shlex.quote(cell)}

## begin project 7 work

#### 1) Unique node labels and the number of nodes for each

In [8]:
%%neo
MATCH (n) RETURN distinct labels(n), count(n)

+---------------------------+
| labels(n)      | count(n) |
+---------------------------+
| ["Recipe"]     | 11634    |
| ["Author"]     | 303      |
| ["Ingredient"] | 3077     |
| ["DietType"]   | 12       |
| ["Collection"] | 1049     |
+---------------------------+

5 rows
ready to start consuming query after 3 ms, results consumed after another 10 ms


#### 2) Unique relationship types and the number of relationships for each

In [9]:
%%neo
MATCH () -[r]-> () RETURN distinct type(r), count(r)

+----------------------------------+
| type(r)               | count(r) |
+----------------------------------+
| "WROTE"               | 11634    |
| "COLLECTION"          | 32240    |
| "CONTAINS_INGREDIENT" | 106148   |
| "DIET_TYPE"           | 14858    |
+----------------------------------+

4 rows
ready to start consuming query after 39 ms, results consumed after another 63 ms


#### 3) Recipes that contain both chocolate and vanilla

In [19]:
%%neo
Match (r:Recipe) -[]-> (i:Ingredient), 
(r) -[]-> (i2:Ingredient) 
where i.name = 'chocolate' and 
i2.name = 'vanilla' 
RETURN r.name as recipe, r.description as description

+----------------------------------------------------------------------------------------------------------------------------------------------------------+
| recipe                            | description                                                                                                          |
+----------------------------------------------------------------------------------------------------------------------------------------------------------+
| "Microwave chocolate cake"        | "Edwina Hanslo's delicious and super quick chocolate cake with a creamy ganache is the perfect choice for busy mums" |
| "White chocolate crÃ¨me brÃ»lÃ©e" | "Serve this super-creamy chocolate dessert on its own, or as part of an 'assiete' of mini chocolate puddings"        |
| "Pears with chocolate sauce"      | "A good recipe for chocolate sauce is invaluable. Put everything out on the table and let everyone help themselves"  |
| "Pears with speedy choc sauce"    | "This super-quick pu

#### 4) Good Food recipes sorted by cooking time in descending order

In [11]:
%%neo
MATCH (a:Author) -[]-> (r:Recipe) 
WHERE a.name = 'Good Food' 
RETURN a.name as author, r.name as recipe, r.cookingTime as cookingTime 
ORDER BY r.cookingTime DESC 
LIMIT 20

+------------------------------------------------------------------------------------------------------------+
| author      | recipe                                                                         | cookingTime |
+------------------------------------------------------------------------------------------------------------+
| "Good Food" | "Pork & peach kebabs with Little Gem salad"                                    | 36720       |
| "Good Food" | "Texas barbecue brisket"                                                       | 21600       |
| "Good Food" | "Spiced duck & date tagine"                                                    | 19200       |
| "Good Food" | "Lamb kleftiko"                                                                | 19200       |
| "Good Food" | "Beef strips with crunchy Thai salad"                                          | 18360       |
| "Good Food" | "Roast turkey with sage & onion butter and Marsala gravy"                      | 17100       |
|

#### 5) Challenging dishes that have no preparation time

In [12]:
%%neo
MATCH () -[]-> (r:Recipe)
WHERE r.preparationTime = 0 and
r.skillLevel = 'A challenge'
RETURN r.name as recipe, r.description as description
LIMIT 100

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| recipe                                       | description                                                                                                                                                                                                                                 |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| "Damson jelly"                               | "A delicious way to deal with a glut of damsons - perfect on toast or fresh bread and a gr

#### 6) Easy vegetarian or vegan recipes that take longer to prepare than cook sorted alphabetically

In [14]:
%%neo
MATCH (r:Recipe) -[]-> (d:DietType)
WHERE d.name in ['Vegetarian', 'Vegan'] and
r.preparationTime > r.cookingTime and r.skillLevel = 'Easy'
RETURN r.name as recipe, r.preparationTime as preparationTime, r.cookingTime as cookingTime,d.name as diet
ORDER BY r.name
LIMIT 20

+-------------------------------------------------------------------------------------------+
| recipe                                     | preparationTime | cookingTime | diet         |
+-------------------------------------------------------------------------------------------+
| " Camomile tea with honey "                | 300             | 0           | "Vegetarian" |
| " Pastry snakes"                           | 1620            | 840         | "Vegetarian" |
| " Quick banana ice cream sandwiches"       | 900             | 720         | "Vegetarian" |
| " Red onion with peanut butter & chilli"   | 300             | 0           | "Vegan"      |
| "10-minute couscous salad"                 | 600             | 0           | "Vegetarian" |
| "10-minute mincemeat"                      | 600             | 0           | "Vegetarian" |
| "5-a-day burger"                           | 3600            | 900         | "Vegetarian" |
| "5-minute mocha pots"                      | 300          

#### 7) Gordon Ramsay's vegetarian recipes that do not contain olive oil

In [15]:
%%neo
MATCH (a:Author) -[]-> (r:Recipe),
(r) -[]-> (i:Ingredient),
(r) -[]-> (d:DietType)
WHERE a.name = 'Gordon Ramsay' and
i.name <> 'olive oil' and
d.name = 'Vegetarian'
RETURN distinct a.name as author, r.name as recipe, d.name as diet

+-----------------------------------------------------------------------------------+
| author          | recipe                                           | diet         |
+-----------------------------------------------------------------------------------+
| "Gordon Ramsay" | "Cherry tomatoes"                                | "Vegetarian" |
| "Gordon Ramsay" | "Beets, leeks & onions"                          | "Vegetarian" |
| "Gordon Ramsay" | "Pear & saffron compote"                         | "Vegetarian" |
| "Gordon Ramsay" | "Aubergine timbales with goat's cheese"          | "Vegetarian" |
| "Gordon Ramsay" | "Risotto of spring vegetables"                   | "Vegetarian" |
| "Gordon Ramsay" | "Hot chocolate soufflÃ© "                        | "Vegetarian" |
| "Gordon Ramsay" | "Balsamic beetroot with Roquefort"               | "Vegetarian" |
| "Gordon Ramsay" | "Stuffed courgette rolls"                        | "Vegetarian" |
| "Gordon Ramsay" | "Lemon tart with summer berries"  

#### 8) List of Aphrodisiac recipes

In [16]:
%%neo
MATCH (r:Recipe)-[r_c:COLLECTION]->(c:Collection)
WHERE c.name = 'Aphrodisiac'
RETURN r.name as recipe, r.description as description, id(r_c), c.name as collection

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| recipe                                                 | description                                                                                                                                                         | id(r_c) | collection    |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| "Rib-eye steak with basil hollandaise"                 | "Gordon takes steak and chips to a new level with a sharing slab of prime rib-eye that's just right for sharing. The perfect Valentine's treat for any steak lover" | 13610   | "Aphrodisiac

#### 9) Recipes not written by Good Food or BBC Good Food Team

In [27]:
%%neo
MATCH (a:Author)-[:WROTE]->(r:Recipe)
WHERE NOT a.name in ['Good Food', 'BBC Good Food Team']
RETURN distinct a.name as author, r.name as recipe
LIMIT 25

+------------------------------------------------------------------------+
| author        | recipe                                                 |
+------------------------------------------------------------------------+
| "Tony Tobin"  | "Pork & parsnip cobbler"                               |
| "Tony Tobin"  | "Ratatouille with goat's cheese & herby crumble"       |
| "Tony Tobin"  | "Beetroot, bean & feta salad"                          |
| "Tony Tobin"  | "Warm chicken salad with garlic mushrooms"             |
| "Tony Tobin"  | "Cranberry sauce with port & star anise"               |
| "Tony Tobin"  | "Red pepper, mozzarella & spinach melts"               |
| "Tony Tobin"  | "Salad of smoked haddock with chorizo and poached egg" |
| "Tony Tobin"  | "Daube of beef with spiced beetroot"                   |
| "Tony Tobin"  | "Cheese & fruit sticks"                                |
| "Tony Tobin"  | "French bread sauce"                                   |
| "Tony Tobin"  | "Proven

#### 10) Healthy recipes in the healthy fish collection written by Good Food

In [39]:
%%neo
MATCH (r:Recipe)-[:DIET_TYPE]->(d:DietType),
(r)-[:COLLECTION]->(c:Collection),
(a:Author)-[:WROTE]->(r)
WHERE d.name = 'Healthy' and
c.name = 'Healthy fish' and
a.name = 'Good Food'
RETURN r.name as recipe_name, r.description as description

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| recipe_name                             | description                                                                                                                                             |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| "Fish parcels with romesco sauce & veg" | "Romesco sauce is made from peppers, almonds, chilli and garlic. Team with white fish and a cauliflower and spring onion mix"                           |
| "Classic chunky fish cakes"             | "Fish cakes are a great freezer staple as they are ready portioned so you can defrost as many as you need"                                              |
| "Baked s

## begin extra credit work

#### 1) Create a new node for each existing node label with at least one property

In [40]:
%%neo
CREATE (:Author {name: "Steven Yoo"}), 
(:Collection {name: "Pasta"}),
(:DietType {name: "Carb"}),
(:Ingredient {name: "Tomato"}),
(:Recipe {preparationTime: 300, name: "Spicy Rigatoni Pasta", description: "A spicy, creamy, rigatoni pasta made with tomato sauce", id: "123456", skillLevel: "Easy", cookingTime: 900})

0 rows
ready to start consuming query after 68 ms, results consumed after another 0 ms
Added 5 nodes, Set 10 properties, Added 5 labels


#### 2) Create a relationship for each existing relationship type with at least one property

In [42]:
%%neo
MATCH (r:Recipe {name: "Spicy Rigatoni Pasta"})
MATCH (c:Collection {name: "Pasta"})
CREATE (r)-[:COLLECTION]->(c)

0 rows
ready to start consuming query after 90 ms, results consumed after another 0 ms
Created 2 relationships


In [43]:
%%neo
MATCH (r:Recipe {name: "Spicy Rigatoni Pasta"})
MATCH (i:Ingredient {name: "Tomato"})
CREATE (r)-[:CONTAINS_INGREDIENT]->(i)

0 rows
ready to start consuming query after 60 ms, results consumed after another 0 ms
Created 1 relationships


In [44]:
%%neo
MATCH (r:Recipe {name: "Spicy Rigatoni Pasta"})
MATCH (d:DietType {name: "Carb"})
CREATE (r)-[:DIET_TYPE]->(d)

0 rows
ready to start consuming query after 57 ms, results consumed after another 0 ms
Created 1 relationships


In [45]:
%%neo
MATCH (a:Author {name: "Steven Yoo"})
MATCH (r:Recipe {name: "Spicy Rigatoni Pasta"})
CREATE (a)-[:WROTE]->(r)

0 rows
ready to start consuming query after 59 ms, results consumed after another 0 ms
Created 1 relationships


#### 3) Update the properties of one or more nodes

In [46]:
%%neo
MATCH (d:DietType {name: "Carb"})
SET d.name = "Carbs"

0 rows
ready to start consuming query after 37 ms, results consumed after another 0 ms
Set 1 properties


#### 4) Update the properties of one or more relationships

In [47]:
%%neo
MATCH (a:Author {name: "Steven Yoo"})
MATCH (r:Recipe {name: "Spicy Rigatoni Pasta"})
MERGE (a)-[w:WROTE]->(r)
on MATCH SET w.date = "11/02/2023"

0 rows
ready to start consuming query after 182 ms, results consumed after another 0 ms
Set 1 properties


#### 5) Add a new node label to an existing node

In [56]:
%%neo
MATCH (d:DietType {name: "Carbs"})
SET d:Macro

0 rows
ready to start consuming query after 45 ms, results consumed after another 0 ms
Added 1 labels


#### 6) Update the relationship type of at least one relationship

In [53]:
%%neo
MATCH (n:Author {name: "Steven Yoo"})-[rel:WROTE]->(p:Recipe {name: "Spicy Rigatoni Pasta"})
MERGE (n)-[:CREATED]->(p)
DELETE rel

0 rows
ready to start consuming query after 92 ms, results consumed after another 0 ms
Created 1 relationships, Deleted 1 relationships


#### 7) Delete at least one relationship based on some criteria (Remove recipes that contain muscovado sugar from the Healthy DietType)

In [63]:
%%neo
MATCH (r:Recipe)-[]->(i:Ingredient), 
(r)-[r_d]->(d:DietType) 
WHERE i.name = 'muscovado sugar' and d.name = 'Healthy' 
DELETE r_d

0 rows
ready to start consuming query after 108 ms, results consumed after another 0 ms
Deleted 8 relationships


#### 8) Delete at least one node based on some criteria (Delete Steven Yoo and his recipes)

In [64]:
%%neo
MATCH (a:Author)-[a_r:CREATED]->(r:Recipe), 
(r)-[r_n]->() 
WHERE a.name = 'Steven Yoo' 
DELETE r_n, a_r, r, a

0 rows
ready to start consuming query after 90 ms, results consumed after another 0 ms
Deleted 2 nodes, Deleted 5 relationships


#### 9) Create a unique constraint on a node property

In [71]:
%%neo
CREATE CONSTRAINT unique_constraint_node_prop
FOR (r:Recipe) 
REQUIRE r.date IS UNIQUE

0 rows
ready to start consuming query after 49 ms, results consumed after another 0 ms
Added 1 constraints


#### 10) Create a unique constraint on a relationship property

In [70]:
%%neo 
CREATE CONSTRAINT unique_constraint_relationship_prop 
FOR () -[w:CREATED]- () 
REQUIRE w.id IS UNIQUE

0 rows
ready to start consuming query after 52 ms, results consumed after another 0 ms
Added 1 constraints
